# 할것들

In [1]:
#필요한것

#현재 거치소 목록

#현재 거치소 위치

#각 거치소의 각시간별 평균 사용량(과거데이터)

#각 거치서 부터 가장 가까운 거치소로부터의 거리 / 반경 400m 내 개수

#lcd인지 qr인지

#lcd라면 또다른 lcd거치소와의 가장 가까운 거리 / 만약 qr이라면 다른 거치대로부터의 거리

#가장 가까운 지하철로 부터의 거리 / 반경 400m 내 개수 (역별 승하차인원정보)

#가장 가까운 버스정류장으로 부터의 거리 / 반경 400m 내 개수

#가장 가까운 학교로부터의 거리 / 반경 400m 내 개수

#가장 가까운 공원으로부터의 거리 / 반경 400m 내 개수 (공원 클러스터링)

#거치소별 거치가능한 대수

#가장 가까운 전통시장으로부터의 거리 / 반경 400m 내 개수 (전통시장 클러스터링)

#거치소가 속한 지역의 인구합계, 연령대별 인구수, 1인가구수, 세대수 (끝)

#거치소가 속한 지역구의 구별 시간별 평균 유동인구수(끝)

####크롤링을 통해 도보로 얼마나 걸리는지? 구하는 크롤러제작

#한강 700m내에 위치 여부

#commute 같은 자치구 내 통근통학 인구 수 연속형(끝)

###house 행정동 내 주택 수 연속형(끝)

#가장 가까운 문화공간으로부터의 거리 / 반경 400m내 개수

###거치소가 속한 자치구의 연령대별 인구수 현황 (끝)




# 함수 선언

In [2]:
def naver_map_crawler(start_spot, finish_spot):
    #perfect 완성본 - visible
    from selenium import webdriver
    import pyautogui
    from bs4 import BeautifulSoup
    import time
    from datetime import datetime
    import time
    start = start_spot
    finish = finish_spot
    data = []

    ## 도보는 50km를 넘는 순간 검색결과가 나오지 않는다

    try:
        options = webdriver.ChromeOptions()
        #options.add_argument("headless")
        options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36")
        driver = webdriver.Chrome('/Users/pym/Desktop/dsstack-3/shared/notebooks/3_web-scraping/chromedriver')
        driver.get('https://map.naver.com/v5/directions/-/-/-/mode?c=14107103.1786139,4494701.9630842,15,0,0,0,dh')
        time.sleep(4)
        driver.find_element_by_xpath('//*[@id="sidebar"]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[2]/a').click()
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/ul/li[3]/a').click()
        time.sleep(0.5)
        driver.find_element_by_id('directionStart0').send_keys(start)
        time.sleep(1)
        pyautogui.press('enter')
        time.sleep(0.5)
        driver.find_element_by_id('directionGoal1').send_keys(finish)
        time.sleep(1)
        pyautogui.press('enter')
        time.sleep(0.5)
        driver.find_element_by_xpath('//*[@id="container"]/shrinkable-layout/div/directions-layout/directions-result/div[1]/div[1]/directions-search/div[2]/button[3]').click()
        time.sleep(1)    
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        score_result = soup.find('div', {'class': 'summary_box'}).get_text()

        score_result = str(score_result)
        times, length = score_result.split("추천", 1)


        #print("거리 : " + length + "   /   소요예상시간 : " + times)
        driver.close()
        return times, length
    except:
        times = '99999'
        length = '99999'
        return times, length

In [2]:
import os

def readcsv_(Path_folder, header = 0):

    file_list = [file for file in sorted(os.listdir(Path_folder)) if file.endswith(('.csv','.xlsx'))] 
    df_list = []
    encodings = ['utf-8', 'cp949', 'euc-kr', 'cp1252', 'cp1252']

    for idx, file in enumerate(file_list):
        print(f'({idx}) {file}')
        if file.endswith(".csv"):
            for cp in encodings:
                try:
                    locals()[f'df{idx}'] = pd.read_csv(Path_folder+file, encoding = cp, header = header)
                    df_list.append(locals()[f'df{idx}'])
                    break
                except: # 에러 종류
                    pass
        elif file.endswith(".xlsx"):
                locals()[f'df{idx}'] = pd.read_excel(Path_folder+file, header = header)
                df_list.append(locals()[f'df{idx}'])

        else:
            print(f'     {"*"*8}이 파일은 읽을 수 없습니다{"*"*8}')

    print(f'\n폴더 내 csv / xlsx 파일 개수: {len(file_list)}개')
    print(f'총 데이터프레임 개수: {len(df_list)}개')

    return df_list

In [4]:
def get_addr(X, Y):
    '''
    X,Y에 경도,위도를 넣으면 주소를 return 한다.
    '''
    apiKey='95B9D6ED-C2DB-3B0A-A43F-20FD442638CF'

    try:
        r =requests.get(f'http://apis.vworld.kr/coord2jibun.do?x={X}&y={Y}\
        &apiKey={apiKey}&domain=http://map.vworld.kr/&output=json')
        location = r.json()
#         print(location)
        result = location['ADDR']
    except:
        print(f'에러:{X},{Y}')
        result = 0
        pass
    return result

# 시작

In [3]:
import numpy as np
import pandas as pd
import requests

now_bike_station = pd.read_csv('./now_bike_station_list.csv')

now_bike_station['주소'] = now_bike_station.apply(lambda x: get_addr(x['경도'], x['위도']) if x['주소'] == 0 \
                                     else x['주소'], axis = 1)

now_bike_station.iloc[302,3] = "서울특별시 성북구 장위3동"
now_bike_station.iloc[1070,3] = "서울특별시 강남구 대치4동 1011-31"
now_bike_station.iloc[2024,3] = "서울특별시 은평구 진흥로"
now_bike_station.iloc[2025,3] = "서울특별시 은평구 녹번동 115-2"

now_bike_station['자치구'] = now_bike_station.주소.apply(lambda x : x.split(' ')[1])

bike_location = now_bike_station[now_bike_station.주소.str.contains('서울특별시')]

#bike_location.to_csv('./bike_location_final.csv', index=False)

bike_location.reset_index(drop=True, inplace=True)

# 각 거치소로부터 다른 거치소 사이의 거리

In [6]:
bike_location= pd.read_csv('./bike_location_final.csv')

bike_location['대여소번호'] = bike_location.대여소이름.apply(lambda x : int(x.split('.')[0]))

In [7]:
bike_location

,대여소이름,위도,경도,주소,자치구,대여소번호
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003
3,1004. 삼성광나루아파트 버스정류장,37.553329,127.128868,서울특별시 강동구 암사동 463-2,강동구,1004
4,1006. 롯데캐슬 115동앞,37.554867,127.142799,서울특별시 강동구 암사동 414-2,강동구,1006
...,...,...,...,...,...,...
2038,992.북한산입구 정류장,37.658428,126.946060,서울특별시 은평구 진관동 289,은평구,992
2039,993.은평뉴타운 온뜨락아파트앞,37.630287,126.919563,서울특별시 은평구 진관동 234-34,은평구,993
2040,994.북한산생태공원입구(교통섬),37.610508,126.938538,서울특별시 은평구 불광동 2-1,은평구,994
2041,995.은평다목적체육관,37.631409,126.925407,서울특별시 은평구 진관동 125,은평구,995


### 거치소 형식 가져다 붙여버렷 ㅇㅁ니ㅏ러ㅣㅁ아너히ㅏㅁㅇ너


In [8]:
bikelock = readcsv_('/Users/pym/data/ddareung/자전거보관소_정보/')

Bikelock = bikelock[0]

Bikelock_1 = Bikelock[['대여소 번호', '보관소(대여소)명', '자치구', '상세주소', '운영 방식']]

Bikelock_1.columns = ['대여소번호','보관소명','자치구','상세주소','운영방식']

Bikelock_2 = Bikelock_1[['대여소번호','운영방식']] 

bike_location_alpha = pd.merge(bike_location, Bikelock_2, on=['대여소번호'], how='left')

bike_location_alpha.운영방식.fillna('QR', inplace=True)

bike_location_alpha.loc[bike_location_alpha.대여소이름 == '583. 청계천 생태교실 앞', '운영방식'] = 'QR'

bike_location_alpha.drop_duplicates(inplace=True)

(0) 공공자전거 대여소 정보(20.07.13 기준) .csv

폴더 내 csv / xlsx 파일 개수: 1개
총 데이터프레임 개수: 1개


In [9]:
bike_location_alpha

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD
3,1004. 삼성광나루아파트 버스정류장,37.553329,127.128868,서울특별시 강동구 암사동 463-2,강동구,1004,LCD
4,1006. 롯데캐슬 115동앞,37.554867,127.142799,서울특별시 강동구 암사동 414-2,강동구,1006,LCD
...,...,...,...,...,...,...,...
2039,992.북한산입구 정류장,37.658428,126.946060,서울특별시 은평구 진관동 289,은평구,992,QR
2040,993.은평뉴타운 온뜨락아파트앞,37.630287,126.919563,서울특별시 은평구 진관동 234-34,은평구,993,QR
2041,994.북한산생태공원입구(교통섬),37.610508,126.938538,서울특별시 은평구 불광동 2-1,은평구,994,QR
2042,995.은평다목적체육관,37.631409,126.925407,서울특별시 은평구 진관동 125,은평구,995,QR


for data, data_1 in zip(bike_location_alpha.iloc[:,:-1].iterrows(), bike_location.iterrows()):
    if "".join(list(str(data))).replace(' ', '').replace('\n', '') != "".join(list(str(data_1))).replace(' ', '').replace('\n', ''):
        print("".join(list(str(data))).replace(' ', '').replace('\n', ''), "".join(list(str(data_1))).replace(' ', '').replace('\n', ''), '\n')
    
    

# 세대수

In [11]:
house_count = pd.read_csv('./new_data/Cencus2019.csv')

In [12]:
house_count = house_count[['자치구','세대']]

# 자치구연령대별인구수

In [13]:
age_population = pd.read_csv('./new_data/자치구연령대별인구수.csv', encoding='cp949')

In [14]:
age_population = age_population.iloc[1:,:]

In [15]:
age_population.행정구역 = age_population.행정구역.apply(lambda x : x.split('(')[0].split(' ')[1])

In [16]:
age_population.columns=['자치구','거주자총인구수','drop', '0s','10s','20s','30s','40s','50s','60s','70s','80s','90s','100s']

In [17]:
age_population.drop(['drop', '100s'], axis=1, inplace=True)

In [18]:
age_population #merge하면 끝

,자치구,거주자총인구수,0s,10s,20s,30s,40s,50s,60s,70s,80s,90s
1,종로구,"144,313","7,649","11,048","23,145","19,687","21,569","24,466","18,661","11,693","5,535",833
2,중구,"121,505","6,931","6,841","18,439","19,882","17,582","19,973","16,587","10,103","4,447",706
3,용산구,"220,118","13,568","15,610","32,958","37,370","34,359","34,987","26,586","15,791","7,679","1,178"
4,성동구,"289,614","20,658","20,030","45,005","48,735","44,488","46,250","35,267","19,960","8,160","1,032"
5,광진구,"342,484","19,884","26,120","62,495","55,688","51,980","53,736","41,156","21,801","8,476","1,106"
6,동대문구,"337,218","20,984","23,853","56,005","49,764","49,907","52,048","44,919","27,381","10,979","1,331"
7,중랑구,"390,106","23,389","27,005","55,893","57,839","57,225","68,512","56,668","30,765","11,340","1,432"
8,성북구,"431,391","29,209","37,397","66,624","59,748","69,014","67,913","53,273","32,732","13,646","1,802"
9,강북구,"303,859","16,051","22,226","42,655","38,608","46,207","51,643","44,646","29,383","11,088","1,325"
10,도봉구,"322,436","19,069","25,835","43,405","40,877","48,215","56,335","48,584","27,259","11,285","1,523"


# 1인가구수

In [19]:
one_person_house = pd.read_csv('./new_data/1인가구수.csv', encoding='cp949')

In [20]:
one_person_house.columns = ['자치구','성별','가구수']

In [21]:
one_person_house.drop('성별', axis=1,inplace=True)

In [22]:
one_person_house = one_person_house.iloc[2:,:]

In [23]:
one_person_house.columns = ['자치구','1인가구수']

# 지하철 승차인원

In [9]:
subway_population = readcsv_('./data/지하철승하차인원/')

(0) CARD_SUBWAY_MONTH_202002.csv
(1) CARD_SUBWAY_MONTH_202003.csv
(2) CARD_SUBWAY_MONTH_202004.csv
(3) CARD_SUBWAY_MONTH_202005.csv
(4) CARD_SUBWAY_MONTH_202006.csv
(5) CARD_SUBWAY_MONTH_202007.csv
(6) CARD_SUBWAY_MONTH_202008.csv
(7) CARD_SUBWAY_MONTH_202009.csv
(8) CARD_SUBWAY_MONTH_202010.csv

폴더 내 csv / xlsx 파일 개수: 9개
총 데이터프레임 개수: 9개


In [10]:
for i in range(9):
    print(subway_population[i].columns)

Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역ID', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역ID', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역ID', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역ID', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')
Index(['사용일자', '노선명', '역ID', '역명', '승차총승객수', '하차총승객수', '등록일자'], dtype='object')


In [26]:
subway_population[3] = subway_population[3].reset_index()

subway_population[3].columns = ['사용일자','노선명','역명','승차총승객수','하차총승객수','등록일자','드롭']

subway_population[3].drop('드롭', axis=1, inplace=True)

In [11]:
df_1 = pd.concat([subway_population[0],subway_population[1],subway_population[2],subway_population[3]],axis=0)

In [12]:
df_1.drop('등록일자', axis=1, inplace=True)

In [13]:
df_2 = pd.concat([subway_population[4],subway_population[5],subway_population[6],subway_population[7],
           subway_population[8]], axis=0)

In [14]:
df_2.columns = ['사용일자','노선명','역명','승차총승객수','하차총승객수','등록일자','드랍']

In [15]:
df_2.drop(['등록일자','드랍'],axis=1, inplace=True)

In [16]:
subway_people = pd.concat([df_1, df_2], axis=0)

In [17]:
subway_people.reset_index(drop=True, inplace=True)

In [18]:
subway_people.사용일자 = subway_people.사용일자.astype(str)

In [19]:
subway_people.사용일자 = subway_people.사용일자.apply(lambda x : str(x[:4])+'-'+str(x[4:6])+'-'+str(x[6:9]))

In [36]:
import datetime
subway_people.사용일자 = subway_people.사용일자.apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))

In [37]:
subway_people['weekday'] = subway_people.사용일자.apply(lambda x : x.weekday())

In [38]:
subway_people['평일_주말'] = subway_people.weekday.apply(lambda x : '평일' if x <5 else '주말')

In [39]:
subway_people['승하차총승객수'] =subway_people['승차총승객수'] + subway_people['하차총승객수'] 

In [40]:
subway_people_mean = subway_people.groupby(['역명','노선명','평일_주말'],as_index=False)[['승하차총승객수']].mean()

In [41]:
subway = pd.read_csv('./new_data/Subway.csv', encoding='cp949')

In [42]:
subway.역명 = subway.역명.apply(lambda x : x.split(' ')[0])

In [43]:
subway.loc[subway.선명.str.contains('선') == False,'선명'] = \
subway[subway.선명.str.contains('선') == False].선명.apply(lambda x : x + '선')

In [44]:
subway_people_mean.노선명 = \
subway_people_mean.노선명.apply(lambda x :'경의중앙선' if (x == '경의선') | (x == '중앙선') else x)

In [45]:
subway_people_mean.노선명 = \
subway_people_mean.노선명.apply(lambda x :'공항철도' if x=='공항철도 1호선' else x)

In [46]:
subway.columns = ['지번주소', '역명', '노선명', '위도','경도']

In [47]:
subway.노선명 = \
subway.노선명.apply(lambda x :'공항철도' if x=='공항선' else x)

In [48]:
df_1 = pd.read_excel('./new_data/신분당선 주소데이터.xlsx')

In [49]:
df_1.drop('철도운영기관명', axis =1, inplace=True)

In [50]:
df_1 = df_1[df_1.지번번호.str.contains('서울')]

In [51]:
df_1.선명 = '신분당선'

In [52]:
df_1.columns = ['노선명','역명','지번주소','도로명주소']

In [53]:
df_1.drop('도로명주소',axis=1,inplace=True)

In [54]:
df_1['위도'] = 0
df_1['경도'] = 0

In [55]:
df_1

,노선명,역명,지번주소,위도,경도
0,신분당선,강남,서울특별시 강남구 역삼동 858,0,0
1,신분당선,양재(서초구청),서울특별시 서초구 서초동 1366-9,0,0
2,신분당선,양재시민의숲(매헌),서울특별시 서초구 양재동 233-2,0,0
3,신분당선,청계산입구,서울특별시 서초구 신원동 270-25,0,0


In [56]:
df_2 = pd.read_excel('./new_data/우이신설 역위치.xlsx')

In [57]:
df_2.drop('철도운영기관명',axis=1, inplace=True)

In [58]:
df_2.선명 = df_2.선명.apply(lambda x : x+'선')

In [59]:
df_2.columns = ['노선명','역명','경도','위도']

In [60]:
df_2['지번주소'] = '없음'

In [61]:
df_2 = df_2[['노선명','역명','지번주소','위도','경도']]

In [62]:
df_2

,노선명,역명,지번주소,위도,경도
0,우이신설선,북한산우이,없음,37.663578,127.012320
1,우이신설선,솔밭공원,없음,37.657816,127.013662
2,우이신설선,4.19민주묘지,없음,37.646942,127.009112
3,우이신설선,가오리,없음,37.641892,127.016643
4,우이신설선,화계,없음,37.634207,127.017474
5,우이신설선,삼양,없음,37.626971,127.018140
6,우이신설선,삼양사거리,없음,37.621065,127.020573
7,우이신설선,솔샘,없음,37.620154,127.014416
8,우이신설선,북한산보국문,없음,37.611672,127.008451
9,우이신설선,정릉,없음,37.602639,127.013443


In [63]:
subway = subway[['노선명','역명','지번주소','위도','경도']]

In [64]:
SUBWAY = pd.concat([subway, df_1, df_2], axis=0)

In [65]:
set(SUBWAY.노선명.unique().tolist())

{'1호선',
 '2호선',
 '3호선',
 '4호선',
 '5호선',
 '6호선',
 '7호선',
 '8호선',
 '9호선',
 '경의중앙선',
 '경춘선',
 '공항철도',
 '분당선',
 '신분당선',
 '우이신설선'}

In [66]:
set(subway_people_mean.노선명.unique().tolist())-set(SUBWAY.노선명.unique().tolist())

{'9호선2~3단계', '경강선', '경부선', '경원선', '경인선', '과천선', '수인선', '안산선', '일산선', '장항선'}

SUBWAY.to_csv('./SUBWAY.csv', index=False)

In [67]:
Subway = pd.read_csv('./new_data/Subway (1).csv',encoding='cp949')

In [68]:
subway_people_mean

,역명,노선명,평일_주말,승하차총승객수
0,4.19민주묘지,우이신설선,주말,3332.253165
1,4.19민주묘지,우이신설선,평일,4855.010256
2,가능,경원선,주말,7631.645570
3,가능,경원선,평일,11621.861538
4,가락시장,3호선,주말,8447.139241
...,...,...,...,...
1219,효창공원앞,6호선,평일,12269.312821
1220,효창공원앞,경의중앙선,주말,2560.898734
1221,효창공원앞,경의중앙선,평일,5460.271795
1222,흑석(중앙대입구),9호선,주말,8006.405063


In [69]:
Subway

,노선명,역명,지번주소,위도,경도
0,1호선,도봉산,서울특별시 도봉구 도봉동 18-1,37.680042,127.039988
1,1호선,도봉,서울특별시 도봉구 도봉동 639,37.675086,127.044867
2,1호선,방학,서울특별시 도봉구 방학동 728,37.669294,127.043858
3,1호선,창동,서울특별시 도봉구 창동 135-1,37.653485,127.047593
4,1호선,녹천,서울특별시 도봉구 창동 57-11,37.639373,127.036705
...,...,...,...,...,...
364,우이신설선,북한산보국문,서울특별시 성북구 정릉동 303-3,37.611672,127.008451
365,우이신설선,정릉,서울특별시 성북구 정릉동 966-330,37.602639,127.013443
366,우이신설선,성신여대입구,서울특별시 성북구 동선동4가 1-3,37.592719,127.016518
367,우이신설선,보문,서울특별시 성북구 보문동1가 127-7,37.585359,127.019409


In [149]:
train = pd.merge(Subway,subway_people_mean, on=['노선명','역명'], how ='left')

In [151]:
train.isna().sum()

노선명         0
역명          0
지번주소        0
위도          0
경도          0
평일_주말      61
승하차총승객수    61
dtype: int64

In [153]:
train[train.평일_주말.isna()]

,노선명,역명,지번주소,위도,경도,평일_주말,승하차총승객수
0,1호선,도봉산,서울특별시 도봉구 도봉동 18-1,37.680042,127.039988,NaN,NaN
1,1호선,도봉,서울특별시 도봉구 도봉동 639,37.675086,127.044867,NaN,NaN
2,1호선,방학,서울특별시 도봉구 방학동 728,37.669294,127.043858,NaN,NaN
3,1호선,창동,서울특별시 도봉구 창동 135-1,37.653485,127.047593,NaN,NaN
4,1호선,녹천,서울특별시 도봉구 창동 57-11,37.639373,127.036705,NaN,NaN
...,...,...,...,...,...,...,...
648,신분당선,강남,서울특별시 강남구 역삼동 858,37.497056,127.028180,NaN,NaN
649,신분당선,양재(서초구청),서울특별시 서초구 서초동 1366-9,37.484799,127.033387,NaN,NaN
650,신분당선,양재시민의숲(매헌),서울특별시 서초구 양재동 233-2,37.470003,127.038325,NaN,NaN
651,신분당선,청계산입구,서울특별시 서초구 신원동 270-25,37.448510,127.054606,NaN,NaN


# 문화공간

In [73]:
culture_place = pd.read_csv('./new_data/서울시 문화공간 현황.csv', encoding='cp949')

In [74]:
culture_place.drop(['대표이미지','전화번호','팩스번호','홈페이지','기타사항','장르분류코드'],axis=1,inplace=True)

In [75]:
culture_place.drop(['문화공간코드','개관일자','관람시간','객석수','관람료(원)','휴관일','무료구분'],axis=1, inplace=True)

In [76]:
culture_place[culture_place.주소.isna()]

,장르분류명,문화공간명,주소,X좌표,Y좌표
50,공연장,PARADISE ZIP,NaN,37.559814505,127.0041650227
54,공연장,용산꿈나무종합타운,NaN,37.5386613022,126.9655079691
56,공연장,롯데콘서트홀,NaN,37.5131010685,127.1034317323
60,공연장,노원에코센터,NaN,37.6441617186,127.0582991353
82,공연장,kt 챔버홀,NaN,37.529459866,126.8711176804
83,공연장,플랫폼 창동 61,NaN,37.6543722563,127.0490690004
227,공연장,서울 암사동 유적,NaN,37.5605464139,127.1302578021
261,박물관/기념관,과천시 추사박물관,NaN,37.4528846,127.0264652
510,문화원,송파마을예술창작소,NaN,37.501586868,127.110680875
744,기타,파주운정행복센터,NaN,37.7240287749,126.7513085673


In [77]:
juso_1 = ["서울특별시 중구 장충동2가 193-5",
"서울특별시 원효로1동",
"서울특별시 송파구 잠실6동",
"서울특별시 노원구 상계6.7동 770-2",
"서울특별시 양천구 목1동 목동서로 201 kt정보전산센터 1층",
"서울특별시 도봉구 창동 1-9",
"서울특별시 암사3동",
"경기도 과천시 주암동 184-2",
"서울특별시 송파구 송파동 지하층 2-13번지",
"경기도 파주시 운정1동 와석순환로 415",
"경기도 하남시 미사동",
"경기도 양평군 단월면 석산리 1",
"서울특별시 중구 정동 세종대로 99",
"서울특별시 동작구 대방동 345-1"]

culture_place.loc[culture_place.주소.isna(), '주소'] = juso_1

In [78]:
culture_place = culture_place[culture_place.주소.str.contains('서울')]

In [79]:
culture_place

,장르분류명,문화공간명,주소,X좌표,Y좌표
0,공연장,신도림 오페라하우스,서울시 구로구 구로동 3-39,37.5079891008,126.8909247643
1,공연장,신촌문화발전소,서울특별시 서대문구 연세로2나길 57,37.5579611736,126.9397535366
2,공연장,서울광장,서울특별시 중구 태평로2가 17-3,37. 565589,126. 978018
3,공연장,오류아트홀,서울특별시 구로구 경인로20가길 38 오류문화센터 2층(오류아트홀),37.495357487,126.8460357988
4,공연장,정효아트센터,서울특별시 서초구 사임당로18길 52-2,37.4875489437,127.0196245765
...,...,...,...,...,...
900,기타,삼청각,서울 성북구 성북동 330-115 ),37.5971248,126.9842404
901,기타,꼬스트홀,서울 중구 명동2가 1-1 ),37.5632657,126.9867360
902,기타,중구청,서울 중구 예관동 120-1 ),37.5638430,126.9976020
903,기타,남산골 한옥마을,서울 중구 필동2가 84-1 ),37.5591447,126.9936826


# 유동인구

In [80]:
floating = pd.read_csv('./new_data/Floating_list.csv')

In [81]:
floating.일자 = floating.일자.apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d'))

In [82]:
floating['weekday'] = floating.일자.apply(lambda x : x.weekday())

In [83]:
floating['평일_주말'] = floating.weekday.apply(lambda x : '평일' if x<5 else '주말')

In [84]:
floating_20_30 = floating[(floating['연령대(10세단위)'] == 20)|(floating['연령대(10세단위)'] == 30)]

In [85]:
moving_people_20_30s = floating_20_30.groupby(['평일_주말','군구','시간(1시간단위)'],as_index=False)[['유동인구수']].mean()

In [86]:
moving_people_20_30s.columns = ['평일_주말','자치구','시간대','20~30대_유동인구'] #20대 시간별 유동인구

In [87]:
moving_people_20_30s

,평일_주말,자치구,시간대,20~30대_유동인구
0,주말,강남구,0,46700.063694
1,주말,강남구,1,45605.222930
2,주말,강남구,2,44934.092357
3,주말,강남구,3,44204.044586
4,주말,강남구,4,43846.114650
...,...,...,...,...
1195,평일,중랑구,19,24238.746787
1196,평일,중랑구,20,25305.880463
1197,평일,중랑구,21,26458.727506
1198,평일,중랑구,22,27612.203608


# 통근통학

In [89]:
commute = pd.read_csv('./new_data/통근통학.csv', encoding='cp949')

In [90]:
commute.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65450 entries, 0 to 65449
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   행정구역별       65450 non-null  object 
 1   성별          65450 non-null  object 
 2   연령별         65450 non-null  object 
 3   통근·통학여부별    65450 non-null  object 
 4   항목          65450 non-null  object 
 5   단위          0 non-null      float64
 6   2015 년      52914 non-null  float64
 7   Unnamed: 7  0 non-null      float64
dtypes: float64(3), object(5)
memory usage: 4.0+ MB


In [91]:
commute.columns

Index(['행정구역별', '성별', '연령별', '통근·통학여부별', '항목', '단위', '2015 년', 'Unnamed: 7'], dtype='object')

In [92]:
commute.drop(['단위','성별','Unnamed: 7'], axis=1, inplace=True)

In [93]:
commute = commute[commute['2015 년'].isna() == False]

In [94]:
commute.행정구역별.unique()

array(['서울특별시', '종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구', '동부',
       '읍부', '면부', '서구', '동구', '영도구', '부산진구', '동래구', '남구', '북구', '해운대구',
       '사하구', '금정구', '연제구', '수영구', '사상구', '기장군', '수성구', '달서구', '달성군',
       '연수구', '남동구', '부평구', '계양구', '강화군', '옹진군', '광산구', '유성구', '대덕구',
       '울주군', '세종시', '수원시', '장안구', '권선구', '팔달구', '영통구', '성남시', '수정구',
       '중원구', '분당구', '의정부시', '안양시', '만안구', '동안구', '부천시', '원미구', '소사구',
       '오정구', '광명시', '평택시', '동두천시', '안산시', '상록구', '단원구', '고양시', '덕양구',
       '일산동구', '일산서구', '과천시', '구리시', '남양주시', '오산시', '시흥시', '군포시', '의왕시',
       '하남시', '용인시', '처인구', '기흥구', '수지구', '파주시', '이천시', '안성시', '김포시',
       '화성시', '광주시', '양주시', '포천시', '여주시', '연천군', '가평군', '양평군', '춘천시',
       '원주시', '강릉시', '동해시', '태백시', '속초시', '삼척시', '홍천군', '횡성군', '영월군',
       '평창군', '정선군', '철원군', '화천군', '양구군', '인제군', '고성군', '양양군', '충주시',
       '제

In [95]:
gu_list = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구',
       '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구',
       '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

In [96]:
commute = commute[commute.행정구역별.isin(gu_list)]

In [97]:
commute = commute[commute['통근·통학여부별'] == '계']

In [98]:
commute.drop('통근·통학여부별', axis=1,inplace=True)

In [99]:
commute = commute[commute.연령별 != '합계']

In [100]:
commute = commute[commute.항목 != '통근·통학 인구(12세 이상)']

In [101]:
commute = commute.groupby(['행정구역별','연령별'], as_index=False)[['2015 년']].sum()

In [102]:
commute.연령별 = commute.연령별.apply(lambda x : "10대" if x[0] == "1" else x)
commute.연령별 = commute.연령별.apply(lambda x : "20대" if x[0] == "2" else x)
commute.연령별 = commute.연령별.apply(lambda x : "30대" if x[0] == "3" else x)
commute.연령별 = commute.연령별.apply(lambda x : "40대" if x[0] == "4" else x)
commute.연령별 = commute.연령별.apply(lambda x : "50대" if x[0] == "5" else x)
commute.연령별 = commute.연령별.apply(lambda x : "60대" if x[0] == "6" else x)
commute.연령별 = commute.연령별.apply(lambda x : "70대이상" if x[0] == "7" else x)

In [103]:
commute

,행정구역별,연령별,2015 년
0,강남구,10대,15138.0
1,강남구,10대,26660.0
2,강남구,20대,15028.0
3,강남구,20대,20626.0
4,강남구,30대,21707.0
...,...,...,...
345,중랑구,50대,13550.0
346,중랑구,60대,7831.0
347,중랑구,60대,4143.0
348,중랑구,70대이상,1836.0


In [104]:
commute = commute[(commute.연령별 == '20대')|(commute.연령별 == '30대')].groupby(['행정구역별','연령별'],\
                                                               as_index=False)[['2015 년']].sum()

In [105]:
commute_20s = commute[commute.연령별 == '20대']
commute_30s = commute[commute.연령별 == '30대']

In [106]:
commute_20s

,행정구역별,연령별,2015 년
0,강남구,20대,35654.0
2,강동구,20대,29744.0
4,강북구,20대,18484.0
6,강서구,20대,41197.0
8,관악구,20대,50437.0
10,광진구,20대,37867.0
12,구로구,20대,26975.0
14,금천구,20대,14705.0
16,노원구,20대,40262.0
18,도봉구,20대,21625.0


In [107]:
commute_30s

,행정구역별,연령별,2015 년
1,강남구,30대,37530.0
3,강동구,30대,30293.0
5,강북구,30대,18443.0
7,강서구,30대,46075.0
9,관악구,30대,42607.0
11,광진구,30대,30045.0
13,구로구,30대,31482.0
15,금천구,30대,14242.0
17,노원구,30대,34448.0
19,도봉구,30대,20296.0


# 거리계산문제

In [108]:
bike_location_alpha

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD
3,1004. 삼성광나루아파트 버스정류장,37.553329,127.128868,서울특별시 강동구 암사동 463-2,강동구,1004,LCD
4,1006. 롯데캐슬 115동앞,37.554867,127.142799,서울특별시 강동구 암사동 414-2,강동구,1006,LCD
...,...,...,...,...,...,...,...
2039,992.북한산입구 정류장,37.658428,126.946060,서울특별시 은평구 진관동 289,은평구,992,QR
2040,993.은평뉴타운 온뜨락아파트앞,37.630287,126.919563,서울특별시 은평구 진관동 234-34,은평구,993,QR
2041,994.북한산생태공원입구(교통섬),37.610508,126.938538,서울특별시 은평구 불광동 2-1,은평구,994,QR
2042,995.은평다목적체육관,37.631409,126.925407,서울특별시 은평구 진관동 125,은평구,995,QR


In [109]:
def walk_meter_x_y(Y,X,Y1,X1):
    url = "https://map.naver.com/v5/api/dir/findwalk?lo=ko&r=step&st=1&o=all&l={},{};{},{}&lang=ko".format(Y,X,Y1,X1)
    
    r = requests.get(url)
    
    import re # 정규표현식 모듈 
 
    p = re.compile('"summary":{"distance":\d+,"duration":\d+}')
    numbers = re.findall("\d+", p.findall(r.text)[0])
    distance = int(numbers[0])#미터단위
    duration = int(numbers[1])/60#초단위
    return distance, duration

In [110]:
def walk_meter_x_y_juso(Y,X,juso,Y1,X1,juso1):
    url = "https://map.naver.com/v5/api/dir/findwalk?lo=ko&r=step&st=1&o=all&l={},{},{};{},{},{}&lang=ko".format(Y,X,juso,Y1,X1,juso1)
    
    r = requests.get(url)
    
    import re # 정규표현식 모듈 
 
    p = re.compile('"summary":{"distance":\d+,"duration":\d+}')
    numbers = re.findall("\d+", p.findall(r.text)[0])
    distance = int(numbers[0])#미터단위
    duration = int(numbers[1])/60   #분단위
    return distance, duration

In [111]:
distance, duration = walk_meter_x_y(126.946060,37.658428,126.938538,37.610508)

# 붙이자!!

bike_location_alpha.to_csv('./bike_location_finally.csv',index=False )

###  자치구별 세대수 컬럼

In [113]:
house_count.columns = ['자치구','세대수']

In [114]:
data_frame = pd.merge(bike_location_alpha, house_count, on='자치구', how='left')

In [115]:
data_frame.head(3)

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식,세대수
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD,"183,390"
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD,"183,390"
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD,"183,390"


### 자치구별 연령대별 인구수

In [116]:
ap_df = age_population[['자치구','거주자총인구수','20s','30s']]

In [117]:
data_frame = pd.merge(data_frame,ap_df, on='자치구', how='left' )

In [118]:
data_frame.head(3)

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식,세대수,거주자총인구수,20s,30s
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD,"183,390","452,350","60,955","71,144"
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD,"183,390","452,350","60,955","71,144"
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD,"183,390","452,350","60,955","71,144"


### 자치구별 1인가구수

In [119]:
data_frame = pd.merge(data_frame,one_person_house, on='자치구', how='left')

In [120]:
data_frame.head(3)

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식,세대수,거주자총인구수,20s,30s,1인가구수
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD,"183,390","452,350","60,955","71,144",45188
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD,"183,390","452,350","60,955","71,144",45188
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD,"183,390","452,350","60,955","71,144",45188


In [121]:
data_frame.isna().sum()

대여소이름      0
위도         0
경도         0
주소         0
자치구        0
대여소번호      0
운영방식       0
세대수        0
거주자총인구수    0
20s        0
30s        0
1인가구수      0
dtype: int64

### 시간대별 유동인구

In [124]:
from tqdm.notebook import tqdm
#평일
moving_people_20_30s_weekday =moving_people_20_30s.loc[moving_people_20_30s['평일_주말'] == '평일'].drop('평일_주말',axis=1)

moving_people_20_30s_weekday.시간대 = moving_people_20_30s_weekday.시간대.apply(lambda x : '평일'+str(x)+'시_유동인구')

moving_people_20_30s_weekday_f = pd.DataFrame()

for jachigu in tqdm(moving_people_20_30s_weekday.자치구.unique().tolist(), desc='진행도'):
    a = moving_people_20_30s_weekday[moving_people_20_30s_weekday.자치구 == jachigu].T.iloc[1:,:]
    a = a.rename(columns=a.iloc[0]).iloc[1:,:]
    a.rename(index={'20~30대_유동인구': jachigu}, inplace=True)
    a_columns = a.columns.tolist()
    a.reset_index(inplace = True)
    a.columns = ['자치구'] + a_columns
    moving_people_20_30s_weekday_f = moving_people_20_30s_weekday_f.append(a)

In [125]:
#주말
moving_people_20_30s_weekend =moving_people_20_30s.loc[moving_people_20_30s['평일_주말'] == '주말'].drop('평일_주말',axis=1)

moving_people_20_30s_weekend.시간대 = moving_people_20_30s_weekend.시간대.apply(lambda x : '주말'+str(x)+'시_유동인구')

moving_people_20_30s_weekend_f = pd.DataFrame()

for jachigu in tqdm(moving_people_20_30s_weekend.자치구.unique().tolist(), desc='진행도'):
    a = moving_people_20_30s_weekend[moving_people_20_30s_weekend.자치구 == jachigu].T.iloc[1:,:]
    a = a.rename(columns=a.iloc[0]).iloc[1:,:]
    a.rename(index={'20~30대_유동인구': jachigu}, inplace=True)
    a_columns = a.columns.tolist()
    a.reset_index(inplace = True)
    a.columns = ['자치구'] + a_columns
    moving_people_20_30s_weekend_f = moving_people_20_30s_weekend_f.append(a)

data_frame = pd.merge(data_frame, moving_people_20_30s_weekday_f, on='자치구',how='left')
data_frame = pd.merge(data_frame, moving_people_20_30s_weekend_f, on='자치구',how='left')

### 통근통학인구

In [126]:
commute_20s.columns =['자치구','연령별','20대_통근통학인구']

commute_30s.columns =['자치구','연령별','30대_통근통학인구']

In [129]:
commute_20s.drop('연령별',axis=1,inplace=True)
commute_30s.drop('연령별', axis=1,inplace=True)

/Users/pym/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [130]:
data_frame = pd.merge(data_frame,commute_20s, on='자치구',how='left')
data_frame = pd.merge(data_frame,commute_30s, on='자치구',how='left')

In [131]:
data_frame.head(3)

,대여소이름,위도,경도,주소,자치구,대여소번호,운영방식,세대수,거주자총인구수,20s,30s,1인가구수,연령별_x,20대_통근통학인구_x,연령별_y,30대_통근통학인구_x,20대_통근통학인구_y,30대_통근통학인구_y
0,1001. 광진교 남단 사거리(디지털프라자앞),37.541794,127.124748,서울특별시 강동구 천호동 355,강동구,1001,LCD,"183,390","452,350","60,955","71,144",45188,20대,29744.0,30대,30293.0,29744.0,30293.0
1,1002. 해공공원(천호동),37.545261,127.125946,서울특별시 강동구 천호동 360,강동구,1002,LCD,"183,390","452,350","60,955","71,144",45188,20대,29744.0,30대,30293.0,29744.0,30293.0
2,1003. 해공도서관앞,37.543957,127.125488,서울특별시 강동구 천호동 360,강동구,1003,LCD,"183,390","452,350","60,955","71,144",45188,20대,29744.0,30대,30293.0,29744.0,30293.0


### 400m내 거치소 개수

In [132]:
bla = bike_location_alpha[['대여소번호','경도','위도','주소','운영방식']]

In [133]:
def walk_meter_x_y(Y,X,Y1,X1):
    url = "https://map.naver.com/v5/api/dir/findwalk?lo=ko&r=step&st=1&o=all&l={},{};{},{}&lang=ko".format(Y,X,Y1,X1)
    
    r = requests.get(url)
    
    import re # 정규표현식 모듈 
 
    p = re.compile('"summary":{"distance":\d+,"duration":\d+}')
    numbers = re.findall("\d+", p.findall(r.text)[0])
    distance = int(numbers[0])#미터단위
    duration = int(numbers[1])/60#초단위
    return distance, duration

bla.to_csv('./거치소간거리.csv', index=False)

In [1330]:


pd.read_csv('./')


def walk_meter_x_y(Y,X,Y1,X1):
    url = "https://map.naver.com/v5/api/dir/findwalk?lo=ko&r=step&st=1&o=all&l={},{};{},{}&lang=ko".format(Y,X,Y1,X1)
    
    r = requests.get(url)
    
    import re # 정규표현식 모듈 
 
    p = re.compile('"summary":{"distance":\d+,"duration":\d+}')
    numbers = re.findall("\d+", p.findall(r.text)[0])
    distance = int(numbers[0])#미터단위
    duration = int(numbers[1])/60#초단위
    return distance, duration




from bs4 import BeautifulSoup
import time
from datetime import datetime
import requests
from tqdm.notebook import tqdm
from scipy.spatial import distance
total_length_time_bike_location = pd.DataFrame()
for i in tqdm(range(len(bla)), desc='진행도'):
    for j in tqdm(range(i+1, 2043),desc='진행도'):
        if i+1 <= 2043:
            if bla.iloc[i, 4] != 'LCD':
                length,time = walk_meter_x_y(float(bla.iloc[i, 1]), float(bla.iloc[i, 2]),float(bla.iloc[j, 1]),float(bla.iloc[j, 2]))
                bk_x = bla.iloc[i, 0]
                bk_y = bla.iloc[j, 0]
                total_length_time_bike_location = total_length_time_bike_location.append(pd.DataFrame({'대여소번호_x':[bk_x],'대여소번호_y':[bk_y],'거리':[length],'시간':[time]}))
                
            elif bla.iloc[i, 4] == 'LCD':
                length,time=walk_meter_x_y(float(bla.iloc[i, 1]), float(bla.iloc[i, 2]),float(bla.iloc[j, 1]),float(bla.iloc[j, 2]))
                if bla.iloc[j, 4] == 'LCD':
                   

                    bk_x = bla.iloc[i, 0]
                    bk_y = bla.iloc[j, 0]
        
                    total_length_time_bike_location = total_length_time_bike_location.append(pd.DataFrame({'대여소번호_x':[bk_x],'대여소번호_y':[bk_y],'거리':[length],'시간':[time]}))

        
        else:
            pass
total_length_time_bike_location

LCD
1001 1002 400 6.0
   대여소번호_x  대여소번호_y   거리   시간
0     1001     1002  400  6.0
LCD
1001 1003 249 3.7333333333333334
   대여소번호_x  대여소번호_y   거리        시간
0     1001     1002  400  6.000000
0     1001     1003  249  3.733333
LCD
1001 1004 1357 20.35
   대여소번호_x  대여소번호_y    거리         시간
0     1001     1002   400   6.000000
0     1001     1003   249   3.733333
0     1001     1004  1357  20.350000
LCD
1001 1006 2620 39.28333333333333
   대여소번호_x  대여소번호_y    거리         시간
0     1001     1002   400   6.000000
0     1001     1003   249   3.733333
0     1001     1004  1357  20.350000
0     1001     1006  2620  39.283333
LCD
1001 1007 2088 32.31666666666667
   대여소번호_x  대여소번호_y    거리         시간
0     1001     1002   400   6.000000
0     1001     1003   249   3.733333
0     1001     1004  1357  20.350000
0     1001     1006  2620  39.283333
0     1001     1007  2088  32.316667
LCD
1001 1008 915 13.716666666666667
   대여소번호_x  대여소번호_y    거리         시간
0     1001     1002   400   6.000000
0     1001 

KeyboardInterrupt: 

### 문화공간과의 거리

In [146]:
culture_place.X좌표 = culture_place.X좌표.apply(lambda x : x.replace(' ',''))
culture_place.Y좌표 = culture_place.Y좌표.apply(lambda x : x.replace(' ',''))

In [136]:
bla

,대여소번호,경도,위도,주소,운영방식
0,1001,127.124748,37.541794,서울특별시 강동구 천호동 355,LCD
1,1002,127.125946,37.545261,서울특별시 강동구 천호동 360,LCD
2,1003,127.125488,37.543957,서울특별시 강동구 천호동 360,LCD
3,1004,127.128868,37.553329,서울특별시 강동구 암사동 463-2,LCD
4,1006,127.142799,37.554867,서울특별시 강동구 암사동 414-2,LCD
...,...,...,...,...,...
2039,992,126.946060,37.658428,서울특별시 은평구 진관동 289,QR
2040,993,126.919563,37.630287,서울특별시 은평구 진관동 234-34,QR
2041,994,126.938538,37.610508,서울특별시 은평구 불광동 2-1,QR
2042,995,126.925407,37.631409,서울특별시 은평구 진관동 125,QR


In [147]:
from bs4 import BeautifulSoup
import time
from datetime import datetime
import requests
from tqdm.notebook import tqdm
from scipy.spatial import distance

bike_loc_to_culture = pd.DataFrame()

for i in tqdm(range(len(bla)), desc='진행도'):
    for j in tqdm(range(len(culture_place)),desc='진행도'):
        
        length,time = walk_meter_x_y(float(bla.iloc[i, 1]), float(bla.iloc[i, 2]),float(culture_place.iloc[j, 4]),float(culture_place.iloc[j, 3]))
        bk_x = bla.iloc[i, 0] #거치소
        cul_y = culture_place.iloc[j, 1]
        bike_loc_to_culture = bike_loc_to_culture.append(pd.DataFrame({'대여소번호_x':[bk_x],'대여소번호_y':[cul_y],'거리':[length],'시간':[time]}))
        
bike_loc_to_culture

KeyboardInterrupt: 